## Unzip finetuning files containing training scripts

In [1]:
!unzip finetuning.zip

Archive:  finetuning.zip
   creating: finetuning/evaluation/
   creating: finetuning/training/
   creating: finetuning/training/configs/
  inflating: finetuning/training/configs/chronos-t5-large.yaml  
  inflating: finetuning/training/configs/chronos-t5-small.yaml  
  inflating: finetuning/training/train.py  
  inflating: finetuning/evaluation/agg-relative-score.py  
  inflating: finetuning/evaluation/evaluate.py  
   creating: finetuning/evaluation/configs/
  inflating: finetuning/evaluation/configs/zero-shot.yaml  
  inflating: finetuning/evaluation/configs/in-domain.yaml  


## Install required packages

In [2]:
!pip install git+https://github.com/amazon-science/chronos-forecasting.git
!pip install "gluonts[torch]"
!pip install typer-config
!pip install triton

  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-req-build-k0zmwlrw
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-req-build-k0zmwlrw
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit 751f5f5f342d31249d6fea65fc0a4abd2f47d3c6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 21.0 MB/s eta 0:00:00
  Created wheel for chronos-forecasting: filename=chronos_forecasting-1.4.1-py3-none-any.whl size=28756 sha256=d5dc5b2abf7c97e21c6a5f0d7e761f0e5800232a121505bb013dbc4e75af7ada
  Stored in directory: /tmp/pip-ephem-wheel-cache-i5u5hx74/wheels/bf/c1/65/08857e57345ef1b34ba0edb4791f1b2594943e82f34e93a4ab
Successfully built chronos-forecasting
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.

## Generate training and validation splits

In [3]:
import pandas as pd
import numpy as np

In [77]:
df = pd.read_csv("Mastercard_stock_history.csv") # change file path as needed
df = df[["Date", "Close"]]
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date")
df = df.resample("D").ffill()

dates = df.index.to_numpy()
df = df[["Close"]]
df = df.to_numpy()

In [78]:
df = [df.squeeze()]

## Actual code

In [81]:
from pathlib import Path
from typing import List, Union

import numpy as np
from gluonts.dataset.arrow import ArrowWriter


def convert_to_arrow(
    path: Union[str, Path],
    time_series: Union[List[np.ndarray], np.ndarray],
    dates: np.ndarray,
    compression: str = "lz4",
):
    """
    Store a given set of series into Arrow format at the specified path.

    Input data can be either a list of 1D numpy arrays, or a single 2D
    numpy array of shape (num_series, time_length).
    """
    assert isinstance(time_series, list) or (
        isinstance(time_series, np.ndarray) and
        time_series.ndim == 2
    )

    start = dates[0]

    dataset = [
        {"start": start, "target": ts} for ts in time_series
    ]

    ArrowWriter(compression=compression).write_to_file(
        dataset,
        path=path,
    )


if __name__ == "__main__":
    time_series = df

    # Convert to GluonTS arrow format
    convert_to_arrow("./mastercard.arrow", time_series=df, dates=dates)

In [83]:
!CUDA_VISIBLE_DEVICES=0 python ./finetuning/training/train.py --config ./finetuning/training/configs/chronos-t5-small.yaml

2024-12-15 13:25:17.060021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 13:25:17.080675: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 13:25:17.086860: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 13:25:17.101451: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-15 13:25:18.323829: W tensorflow/compiler/tf2

Zip the finetuning files modified in this run

In [84]:
!zip -r finetuning.zip finetuning

updating: finetuning/evaluation/ (stored 0%)
updating: finetuning/training/ (stored 0%)
updating: finetuning/ (stored 0%)
updating: finetuning/training/configs/ (stored 0%)
updating: finetuning/training/configs/chronos-t5-large.yaml (deflated 40%)
updating: finetuning/training/configs/chronos-t5-small.yaml (deflated 39%)
updating: finetuning/training/train.py (deflated 71%)
updating: finetuning/evaluation/agg-relative-score.py (deflated 63%)
updating: finetuning/evaluation/evaluate.py (deflated 68%)
updating: finetuning/evaluation/configs/ (stored 0%)
updating: finetuning/evaluation/configs/zero-shot.yaml (deflated 86%)
updating: finetuning/evaluation/configs/in-domain.yaml (deflated 81%)
